In [1]:
import numpy as np
import pyaudio
import cv2
import time

### 設定

In [2]:
# サンプリングレートとチャンクサイズの設定
RATE = 44100
CHUNK = 1024

# PyAudioの初期化
p = pyaudio.PyAudio()

# 左マイクと右マイクのストリームを作成
stream_left = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, input_device_index=0, frames_per_buffer=CHUNK)
stream_right = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, input_device_index=1, frames_per_buffer=CHUNK)

### 関数定義

In [3]:
def get_audio_data(stream):
    """
    音声データの取得\n
    parameters 
    ---
    stream _Stream : camera frame object
    """
    audio_data = stream.read(CHUNK)
    audio_data = np.frombuffer(audio_data, dtype=np.int16)
    return audio_data

def determine_sound_direction(left_data, right_data):
    """音源の方向を特定\n
    prameters
    ---
    left_data audio1 : audio data
    right_data audio2 : audio data
    """
    # 左右の信号の相関を計算
    correlation = np.correlate(left_data, right_data, 'full')
    delay = np.argmax(correlation) - (len(right_data) - 1)
    # 音源がどちらにあるかを判定
    if delay > 0: return "Left", delay
    elif delay < 0: return "Right", delay
    else: return "Center", delay
    
def calculate_decibels(audio_data):
    """デシベルを計算\n
    Parameters
    ---
    audio_data numpy array : audio data
    """
    rms = np.sqrt(np.mean(np.square(audio_data)))
    decibels = 20 * np.log10(rms) if rms > 0 else -np.inf
    return decibels

def calculate_position(frame, direction, delay, mic_distance=0.2):
    """音源の位置を計算
    parameters
    ---
    frame ndarray : camera frame
    direction str : sound direction
    delay int : sound delay
    mic_distance : float : distance between microphones (meters)
    """
    # 音の速度 (m/s)
    sound_speed = 343.2
    
    # 遅延時間 (秒)
    delay_time = delay / RATE
    
    # 距離差 (メートル)
    distance_diff = delay_time * sound_speed
    
    # 音源の位置を推定 (簡略化)
    x = frame.shape[1] // 2
    y = frame.shape[0] // 2
    
    if direction == "Left":
        x -= int(distance_diff / mic_distance * frame.shape[1] // 2)
    elif direction == "Right":
        x += int(distance_diff / mic_distance * frame.shape[1] // 2)
    
    return (x, y)

    # height, width, _ = frame.shape
    # x = width // 2
    # y = height // 2

    # # 左右の位置を調整
    # if direction == "Left":
    #     x -= int(width * 0.25)
    # elif direction == "Right":
    #     x += int(width * 0.25)
    
    # # 簡単な距離の推定（delayに基づく）
    # y += int(delay * 0.1)

    # return (x, y)

In [ ]:
# カメラの初期化
cap = cv2.VideoCapture(0)

try:
    while True:
        # カメラ画像を取得
        ret, frame = cap.read()
        if not ret:
            break
        
        # 左右の音声データを取得
        audio_data_left = get_audio_data(stream_left)
        audio_data_right = get_audio_data(stream_right)
        
        # 音が出ている方向を特定
        direction, delay = determine_sound_direction(audio_data_left, audio_data_right)

        # デシベルを計算
        decibels_left = calculate_decibels(audio_data_left)
        decibels_right = calculate_decibels(audio_data_right)
        
        # 方向に応じてカメラ画像に表示するテキストを設定
        if direction == "Left":
            decibels = decibels_left
        elif direction == "Right":
            decibels = decibels_right
        else:
            decibels = (decibels_left + decibels_right) / 2
        
        # 音源の位置を計算
        position = calculate_position(frame, direction, delay)
        
        # カメラ画像にテキストを描画
        text = f"{decibels:.2f} dB"
        cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # カメラ画像を表示
        cv2.imshow('Camera', frame)
        
        # 'q'キーが押されたらループを終了
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        time.sleep(0.1)

except KeyboardInterrupt:
    print("終了します")

finally:
    # ストリームを閉じる
    stream_left.stop_stream()
    stream_left.close()
    stream_right.stop_stream()
    stream_right.close()
    p.terminate()
    
    # カメラを解放
    cap.release()
    cv2.destroyAllWindows()


In [2]:
c = 1 + 1
c + 2

4